In [1]:
import pandas as pd
import tensorflow as tf
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict
from pyspark.sql.types import *
from pyspark.sql import Row, SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, ArrayType
import cv2
import numpy as np

In [2]:
spark = SparkSession.builder\
    .master("local[4]")\
    .appName("sql.functions tests")\
    .getOrCreate()
sc = spark.sparkContext

In [3]:
#tiff_train_loc = '/Users/johnmorin/KaylaTek/Data/images/train/*.tif'
#tiff_test_loc = '/Users/johnmorin/KaylaTek/Data/images/test/*.tif'
#tiff_train_labels = '/Users/johnmorin/KaylaTek/Data/images/training_data.csv'
#tiff_test_labels = '/Users/johnmorin/KaylaTek/Data/images/test_data.csv'

In [4]:
train_file_loc = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train_labels.csv'
test_file_loc = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test_labels.csv'

In [5]:
TRAINING_FILE_LOCATION = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/*.jpg'
TEST_FILE_LOCATION = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/*.jpg'

In [6]:
#tif_image_df = spark.read.format('image').load('/Users/johnmorin/KaylaTek/Data/images/train/frame_000000.tif')

In [7]:
#to_shape = tf.cast(tf.stack([1024, 1280, 3]), tf.int32)
#image = tf.reshape(tf.decode_raw(encoded_inputs, tf.uint8), to_shape)

In [8]:
def class_text_to_int(row_label):
    if row_label == 'nine':
        return 1
    elif row_label == 'ten':
        return 2
    elif row_label == 'jack':
        return 3
    elif row_label == 'queen':
        return 4
    elif row_label == 'king':
        return 5
    elif row_label == 'ace':
        return 6
    else:
        return None

In [9]:
def load_label_files(label_filename):
    df = pd.read_csv(label_filename)
    grouped = df['filename'].unique()
    
    csv_df = pd.DataFrame()
    for files in grouped:
        mydf = df[df['filename'] == files]
        #print(mydf.head())
        height = int(mydf['height'].unique()[0])
        width = int(mydf['width'].unique()[0])
        filename = mydf.filename.unique()[0]
        label = mydf['class'].tolist()
        #print(list(label))
        num_label = mydf['class'].map(class_text_to_int).tolist()
        xmin = mydf['xmin'].map(lambda x: x/width).tolist()
        ymin = mydf['ymin'].map(lambda x: x/height).tolist()
        xmax = mydf['xmax'].map(lambda x: x/width).tolist()
        ymax = mydf['ymax'].map(lambda x: x/height).tolist()
        temp = pd.DataFrame({'image/filename':filename, 'image/width':width, 'image/height':height, 
                        'image/object/class/label':[num_label], 'image/object/bbox/xmin':[xmin], 
                        'image/object/bbox/ymin':[ymin],'image/object/bbox/xmax':[xmax],
                        'image/object/bbox/ymax':[ymax], 'image/object/class/text':[label],
                        # Change Number of Channels to correct number
                        'image/channels': int(3),
                        #'image/encoded': dataset_util.bytes_feature(encoded_inputs),
                        })
        csv_df = pd.concat([csv_df, temp])
    csv_df.reset_index(inplace= True)
    csv_df.drop('index', axis=1, inplace = True)
    csv_df['image/format'] = 'jpg'
    # making file names the same
    csv_df['image/filename'] = csv_df['image/filename'].str.replace('.JPG', '.jpg')
    return csv_df

In [10]:
def create_tensor(file_loc):
    print(type(file))
    print(file)
    file = file_loc
    img = cv2.imread(file, 1)
    print(img.shape)
    img2 = img[:, :, ::-1]
    inputs_stacked = np.concatenate([img2], axis=-1)
    encoded_inputs = inputs_stacked.tostring()
    # How to open encoded image
    #    decoded = np.frombuffer(encoded_inputs, np.uint8).reshape(height, width, depth)
    #    to_shape = tf.cast(tf.stack([height, width, depth]), tf.int32)
    #    image = tf.reshape(tf.decode_raw(encoded_inputs, tf.uint8), to_shape)
    return encoded_inputs

filter_udf = udf(create_tensor,ByteType())

In [11]:
data_schema = [StructField('image/channels', IntegerType(), True),
               StructField('image/filename', StringType(), True),
               StructField('image/height', IntegerType(), True), 
               StructField('image/object/bbox/xmax', ArrayType(FloatType(), True), True),
               StructField('image/object/bbox/ymax', ArrayType(FloatType(), True), True),
               StructField('image/object/bbox/xmin', ArrayType(FloatType(), True), True),
               StructField('image/object/bbox/ymin', ArrayType(FloatType(), True), True),
               StructField('image/object/class/label', ArrayType(IntegerType(), True), True),
               StructField('image/object/class/text', ArrayType(StringType(), True), True),
               StructField('image/width', IntegerType(), True), 
               StructField('image/format', StringType(), True) 
            ]
final_struc = StructType(fields=data_schema)

In [12]:
#decoded = np.frombuffer(image_df.first().image.data, np.uint8).reshape(image_df.first().image.height, image_df.first().image.width, image_df.first().image.nChannels)
#to_shape = tf.cast(tf.stack([image_df.first().image.height, image_df.first().image.width, image_df.first().image.nChannels]), tf.int32)
#image = tf.reshape(tf.decode_raw(encoded_inputs, tf.uint8), to_shape)

In [13]:
#decoded.shape

In [14]:
# Use OpenCV to read in binary for image file
#cv2_udf = udf(lambda x: cv2.imread(x, 1).tostring(), BinaryType())

#image_df = image_df.withColumn('file_locs', image_df.image.origin.substr(6, 300))

#image_df = image_df.withColumn('cv2', cv2_udf(image_df.file_locs))

In [30]:
def create_tfrecord(pandas_df, image_location, tf_record_loc, which):
    pand_df = sqlContext.createDataFrame(pandas_df)
    image_df = spark.read.format('image').load(image_location)
    if which is 'train':
        image_df = image_df.withColumn('filename', image_df.image.origin.substr(145, 300))
        #print('train')
        #print(image_df.first().filename)
    if which is 'test':
        image_df = image_df.withColumn('filename', image_df.image.origin.substr(144, 300))
        #print('test')
        #print(image_df.first().filename)
    combined_df = image_df.join(pand_df, \
                            image_df.filename == pand_df['image/filename'], \
                            'inner').drop(image_df.filename)
    #print(combined_df.first().image.height)
    #print(combined_df.first()['image/height'])
    # Original Code to add encoded data to 
    #combined_df = combined_df.withColumn('image/encoded', combined_df['image'].data)
    # Code to encode tiffs into binary format when pyspark won't read them
    combined_df = combined_df.withColumn('image/source_id', combined_df.image.origin.substr(6, 300))
    cv2_udf = udf(lambda x: cv2.imread(x, 1).tostring(), BinaryType())
    combined_df = combined_df.withColumn('image/encoded', cv2_udf(combined_df['image/source_id']))
    #print(combined_df.first())
    # Cleaning up dataframe before writing it to tfrecord
    combined_df = combined_df.drop('image')
    
    # Writing Pyspark Dataframe to tfrecord
    combined_df.write.format('tfrecords').option('recordType', 'Example').save(tf_record_loc)
    #combined_df.printSchema()

In [28]:
train_df = load_label_files(train_file_loc)
test_df = load_label_files(test_file_loc)

In [32]:
create_tfrecord(train_df, TRAINING_FILE_LOCATION, 'jpg_card_train', 'train')
create_tfrecord(test_df, TEST_FILE_LOCATION, 'jpg_card_test', 'test')

train
cam_image26.jpg
504
504
test
cam_image7.jpg
540
540


In [ ]:
import os

In [ ]:
dirs = os.listdir('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/')
for file in dirs:
    if 'JPG' in file:
        split = file.split('.')
        #print(file)
        os.rename(os.path.join('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/', file), str(os.path.join('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/', str(split[0]+'.jpg'))))

In [37]:
dirs = os.listdir('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/test/')
count = 0
for file in dirs:
    if '.jpg' in file:
        count += 1
print(count)

67


In [ ]:
#label_files = spark.read.format('csv') \
#    .option("header", "true") \
#    .option("inferSchema", 'true') \
#    .load('train_label_file.csv')

In [38]:
tf_rec = spark.read.format("tfrecords").option("recordType", "Example").load('jpg_card_train/part-r-00000')

In [39]:
tf_rec.printSchema()

root
 |-- image/object/class/text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- image/channels: long (nullable = true)
 |-- image/height: long (nullable = true)
 |-- image/width: long (nullable = true)
 |-- image/object/bbox/xmin: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/object/bbox/ymax: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/filename: string (nullable = true)
 |-- image/format: string (nullable = true)
 |-- image/source_id: string (nullable = true)
 |-- image/object/class/label: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- image/encoded: string (nullable = true)
 |-- image/object/bbox/ymin: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- image/object/bbox/xmax: array (nullable = true)
 |    |-- element: float (containsNull = true)

